# Library Load

In [ ]:
import os
import numpy as np 
import cv2

from tqdm import tqdm

import xml.etree.ElementTree as ET

from PIL import Image

from data_aug.data_aug import *
from data_aug.bbox_util import *

import matplotlib.pyplot as plt 
%matplotlib inline

# Data Load

In [ ]:
# Style Augmentation Image Path
style_aug_img_folder_path = '../data/3. style_aug_img/'
style_aug_img_folder_list = os.listdir(style_aug_img_folder_path)
style_aug_img_folder_list

In [ ]:
# Label Path
label_folder_path = '../data/3. style_aug_label/'
label_folder_list = os.listdir(label_folder_path)
label_folder_list

In [ ]:
# Bounding Box Transform Yolo Shape
def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

In [ ]:
for img_folder_name, label_folder_name in tqdm(zip(img_folder_list, label_folder_list)):
    # 폴더 내의 파일 리스트
    img_list = os.listdir(img_folder_path + img_folder_name)
    label_list = os.listdir(label_folder_path + label_folder_name)
    
    print(img_folder_name, len(img_list))
    print(label_folder_name, len(label_list))
    print('-' * 60)
    
    # 확장자 제거한 임시 레이블 리스트
    tmp_label_list = []
    for label_name in label_list:
        tmp_label = label_name.split('.')[0]
        tmp_label_list.append(tmp_label)
    
    # 레이블링 안된 이미지 제거
    new_img_list = []
    for img_name in img_list:
        tmp_img = img_name.split('.')[0]
        if tmp_img in tmp_label_list:
            new_img_list.append(img_name)
            
    print(img_folder_name, len(new_img_list))
    print(label_folder_name, len(label_list))
    print('=' * 60)
    print('=' * 60)
            
    # XML 파일 파싱
    object_label_list = []
    for label_name_2 in label_list:
        xml_name = label_folder_path + label_folder_name + '/' + label_name_2
        tree = ET.parse(xml_name)
        objects = tree.findall('./object')

        box_list = []
        for object_ in objects:
            x1 = float(object_.findall('./bndbox')[0].find('./xmin').text)
            y1 = float(object_.findall('./bndbox')[0].find('./ymax').text)
            x2 = float(object_.findall('./bndbox')[0].find('./xmax').text)
            y2 = float(object_.findall('./bndbox')[0].find('./ymin').text)
            tmp_box_list = [x1, y1, x2, y2]
            box_list.append(tmp_box_list)

        object_label_list.append(np.array(box_list))
    print('object_label_list', len(object_label_list))
        
    # 이미지 불러오기
    for img_name_2, object_label in zip(new_img_list, object_label_list):
        load_img = cv2.imread(img_folder_path + img_folder_name + '/' + img_name_2)[:,:,::-1]
        plotted_img = draw_rect(load_img, object_label)
        plt.imshow(plotted_img)
        plt.show()

In [ ]:
#plt.imshow(load_img)

In [ ]:
#plt.imshow(img_)

In [ ]:
#plt.imshow(plotted_img)

In [ ]:
"""cv2.imshow('img', plotted_img)
cv2.waitKey()
cv2.destroyAllWindows()"""